# 01. What is machine learning?

- linear regression example
    - plot the data
    - simple regression
 
- training, loss functions, learning rate, batch size, etc.


In [1]:
%matplotlib inline
from sklearn import linear_model
import matplotlib.pyplot as plt